In [ ]:
import os
from database_tools.tools import BuildDatabase

repo_dir = '/home/cam/Documents/database_tools/'
os.chdir(repo_dir)

config = dict(
    low=0.5,                    # low bandpass frequency
    high=8.0,                   # high bandpass frequency
    sim=0.6,                    # similarity threshold
    df=0.2,                     # one sided frequency delta for SNR calculation
    snr_t=2.0,                  # SNR threshold
    hr_diff=1/6,                # maximum heart rate difference between ppg, abp
    f0_low=0.667,               # minimum valid HR in Hz
    f0_high=3.0,                # maximum valid HR in Hz
    abp_min_bounds=[20, 130],   # upper and lower threshold for DBP
    abp_max_bounds=[50, 225],   # upper and lower threshold for SBP
    windowsize=1,               # windowsize for rolling mean
    ma_perc=20,                 # multiplier for peak detection
    beat_sim=0.2,               # lower threshold for beat similarity
)

worker = BuildDatabase(
    output_dir='data-2022-11-08/',
    config=config,
    win_len=256,
    fs=125,
    samples_per_file=2500,
    max_samples=200000,
    data_dir='physionet.org/files/mimic3wdb/1.0/',
)

worker.run()

In [ ]:
import os
import pandas as pd
from database_tools.tools import DataEvaluator

repo_dir = '/home/cam/Documents/database_tools/'
os.chdir(repo_dir)

pd.options.display.max_rows = 2500

df = pd.read_csv('data-2022-11-08/mimic3_stats.csv')
worker = DataEvaluator(stats=df)

figs = worker.run()

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go

sig = []
with open(f'../data-2022-11-08/mimic3/lines/mimic3_0000000.jsonlines', 'r') as f:
    for sample in f:
        sample = json.loads(sample)
        sig.append(sample['abp'])
sig = np.array(sig)

i = 0
x = sig[0, :]

fig = go.FigureWidget()
fig.update_layout(template='plotly_dark')
fig.add_scatter(
    y=x,
)

In [2]:
import vitaldb

caseids = vitaldb.find_cases(['ART', 'PLETH'])
len(caseids)

3459

In [15]:
import plotly.graph_objects as go

data = vitaldb.load_case(caseids[0], ['ART', 'PLETH'], 1/125)

abp = data[:, 0]
ppg = data[:, 1]

In [16]:
len(ppg)

1442638

In [10]:
5770550 / 1154111

4.999995667661083

In [7]:
len(ppg)

1154111

In [17]:
fig = go.FigureWidget()
fig.update_layout(template='plotly_dark')
fig.add_scatter(
    y=ppg[8000:10000],
)

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'e7d24216-7ac6-432b-8a2d-d30e9fc3d0e2',
 …